In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch import nn, optim
import matplotlib.pyplot as plt
import numpy as np


In [1]:
from pathlib import Path

train_dir = Path("data/brain_tumor_dataset/Training")
test_dir = Path("data/brain_tumor_dataset/Testing")


In [ ]:
for class_folder in train_dir.iterdir():
    num_images = len(list(class_folder.glob(*.jpg)))